# EDA – KoGPT Decoder-only 백본

이 노트북은 **KoGPT (decoder-only) 백본 실험용** EDA 전용 노트북입니다.
- KoGPT 토크나이저 기준 토큰 길이 분포
- train/dev vs KoGPT prediction 길이 분포 비교
- 요약 스타일(키워드, 워드클라우드) 확인

공통 EDA 코드는 `dialogue_eda.ipynb`/`eda_kobart.ipynb`에서 복사해 와서 사용하고,
KoGPT 특화 부분(토크나이저, prediction 파일 경로)만 다르게 설정합니다.

## 1. 데이터 로딩

- KoBART/Llama와 동일하게 `train/dev/test`를 불러옵니다.

In [ ]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("data")
train_df = pd.read_csv(DATA_DIR / "train.csv")
dev_df = pd.read_csv(DATA_DIR / "dev.csv")
test_df = pd.read_csv(DATA_DIR / "test.csv")

train_df.head(), dev_df.head(), test_df.head()

## 2. KoGPT 토크나이저 기준 토큰 길이 분포

- KoGPT 토크나이저로 dialogue를 토크나이즈했을 때 토큰 길이 분포를 봅니다.
- KoGPT 입력 길이 제한(max_length)에 대한 여유를 확인합니다.

In [ ]:
# KoGPT 토크나이저 기준 토큰 길이 분포 예시 코드입니다.
# 실제 실험에 사용할 KoGPT 모델 이름으로 교체해서 사용하세요.

from transformers import AutoTokenizer
import matplotlib.pyplot as plt
import seaborn as sns

KOGPT_MODEL_NAME = "kakaobrain/kogpt"  # TODO: 실제 사용 모델명으로 교체
kogpt_tok = AutoTokenizer.from_pretrained(KOGPT_MODEL_NAME)

def kogpt_token_len(text: str) -> int:
    return len(kogpt_tok.encode(str(text), add_special_tokens=True))

train_df["dialogue_len_tok_kogpt"] = train_df["dialogue"].apply(kogpt_token_len)

plt.figure(figsize=(8, 4))
sns.histplot(train_df["dialogue_len_tok_kogpt"], bins=50, kde=True)
plt.title("Train dialogue token length (KoGPT tokenizer)")
plt.xlabel("dialogue_len_tok_kogpt")
plt.ylabel("count")
plt.show()

## 3. KoGPT prediction 길이 분포 vs gold 비교

- KoGPT 기반 decoder-only 실험에서 생성된 prediction CSV를 불러와,
  gold summary vs prediction summary 길이 분포를 비교합니다.
- prediction CSV 경로는 실제 KoGPT 실험 결과 파일로 변경해서 사용하세요.

In [ ]:
PRED_PATH = "prediction/KOGPT_EXPERIMENT.csv"  # TODO: 실제 KoGPT 실험 결과 파일명으로 교체

try:
    pred_df = pd.read_csv(PRED_PATH)
except FileNotFoundError:
    print(f"prediction 파일을 찾을 수 없습니다: {PRED_PATH}")
else:
    gold_len = train_df["summary"].astype(str).str.len()
    pred_len = pred_df["summary"].astype(str).str.len()

    plt.figure(figsize=(8, 4))
    sns.kdeplot(gold_len, label="gold train summary", shade=True)
    sns.kdeplot(pred_len, label="KoGPT pred summary", shade=True)
    plt.title("summary length: gold vs KoGPT prediction")
    plt.xlabel("summary length (chars)")
    plt.legend()
    plt.show()